In [1]:
import pandas as pd
import numpy as np
import datetime
import os
# Add the new folders of Jian later

In [2]:
def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)

write_folder="/home/jian/celery/Append_to_celery_LiveRamp/monthly_update/output_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

week_beginning=last_saturday-datetime.timedelta(days=28)

recent_3_weeks=[last_saturday-datetime.timedelta(days=x*7) for x in range(3)]

In [3]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in recent_3_weeks]
recent_file_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("MasterBiWeekly" in x) & (".txt" in x) ]
    recent_file_list=recent_file_list+list_recent
recent_file_df=pd.DataFrame({"path":recent_file_list,"date":[datetime.datetime.strptime(x.split("MasterBiWeekly")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_file_list]},index=[x for x in range(len(recent_file_list))])


list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("MasterBiWeekly" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
df_1_after_201806=df_1_after_201806[df_1_after_201806['date']>week_beginning]
del df_1_after_201806['index']

new_2_biweekly_files=df_1_after_201806.append(recent_file_df)

print(len(new_2_biweekly_files))
new_2_biweekly_files=new_2_biweekly_files['path'].unique().tolist()

2


In [4]:
if len(new_2_biweekly_files)==2:
    df_all_new_master = pd.DataFrame()
    for file in new_2_biweekly_files:
        df = pd.read_csv(file,nrows = None,sep= '|',usecols = ['customer_id_hashed','email_address_hash','customer_zip_code'],
                         dtype =str)
        df_all_new_master = df_all_new_master.append(df,ignore_index = True)
    print(len(df_all_new_master.index))
    df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
    print(len(df_all_new_master.index))

440423
440423


In [5]:
# Remove previous week ids

all_files_from_SP=list(recursive_file_gen("/home/jian/celery/Append_to_celery_LiveRamp/Previous_Ids_From_Sp/"))
all_files_from_SP=[x for x in all_files_from_SP if "Copy of 48 Stores_727.csv" not in x]
for Sp_file in all_files_from_SP:
    df = pd.read_csv(Sp_file,usecols = ['email_address_hash'])
    previous_email_set=set(df['email_address_hash'].tolist())
    df_all_new_master = df_all_new_master[~df_all_new_master['email_address_hash'].isin(previous_email_set)]
    print(datetime.datetime.now(),len(df_all_new_master.index))



2019-01-02 15:11:54.477794 440415
2019-01-02 15:11:55.060427 440415
2019-01-02 15:11:55.578597 440414
2019-01-02 15:11:56.759163 440411
2019-01-02 15:11:56.920722 440411
2019-01-02 15:11:57.063515 440411
2019-01-02 15:11:57.164023 440411
2019-01-02 15:11:57.391369 440410
2019-01-02 15:12:02.654643 440404
2019-01-02 15:12:03.825948 440404
2019-01-02 15:12:03.943858 440404
2019-01-02 15:12:04.788939 440402
2019-01-02 15:12:05.642445 440400
2019-01-02 15:12:06.177549 440400
2019-01-02 15:12:09.104883 440391
2019-01-02 15:12:10.040544 440391
2019-01-02 15:12:14.186004 440377
2019-01-02 15:12:14.511591 440377
2019-01-02 15:12:14.739704 440377
2019-01-02 15:12:14.892085 440377
2019-01-02 15:12:15.070138 440377
2019-01-02 15:12:15.296827 440375
2019-01-02 15:12:16.836934 440374
2019-01-02 15:12:22.549095 440367
2019-01-02 15:12:24.626808 440363
2019-01-02 15:12:29.939186 440359
2019-01-02 15:12:33.527380 440355
2019-01-02 15:12:34.961897 440354
2019-01-02 15:12:35.522908 440354
2019-01-02 15:

In [6]:
df_all_new_master = df_all_new_master.drop_duplicates('email_address_hash')
df_all_new_master = df_all_new_master.drop_duplicates('customer_id_hashed')
print(len(df_all_new_master.index))

440317


In [7]:
zipmap = pd.read_csv('/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/zip_with_ta_dma.csv',dtype = 'str')
zipmap['zipcodegroup'] = zipmap['revenue_flag']
zipmap = zipmap[['zip','zipcodegroup']].drop_duplicates('zip')
zipmap.columns = ['customer_zip_code','zipcodegroup']
df_all_new_master = pd.merge(df_all_new_master,zipmap,on ='customer_zip_code',how = 'left' )
print(df_all_new_master['zipcodegroup'].unique())
df_all_new_master['zipcodegroup'].fillna('T',inplace = True)
print(df_all_new_master['zipcodegroup'].unique())

last_saturday_str="_"+str(last_saturday).split("-")[1]+str(last_saturday).split("-")[2]+str(last_saturday).split("-")[0][2:]+"_"

for revenue_flag in ['P','S','T']:
    df = df_all_new_master[df_all_new_master['zipcodegroup']==revenue_flag]
    df = df[['customer_id_hashed','email_address_hash','customer_zip_code']]
    df['segment'] = 'NewReward' + last_saturday_str + revenue_flag
    print(revenue_flag,len(df.index))
    df.to_csv(write_folder+'NewReward_'+str(last_saturday)+'_'+revenue_flag+'.csv',index = False)


['S' 'P' nan 'T']
['S' 'P' 'T']
P 308687
S 84154
T 47476
